# Benchmarks and Scaling

We can benchmark the allocation algorithms and determine their behavior with respect
to the size of the fleet and the number of charging posts. We simply need to create
the fleets and charging posts and then run the algorithms via
[timeit](https://docs.python.org/3.8/library/timeit.html), a
timing module from the python standard library.

Lets first define the sizes for which we will run benchmarks. We parametrize the
benchmarks with the size of the fleet and the ratio between the number of charging
posts and the size of the fleet, as well as the underlying algorithm.

In [ ]:
from functools import partial
from typing import Callable, Mapping, Optional

import numpy as np
import pandas as pd

import evosim


def benchmark_set(
    nfleet=(10, 50),
    ratios=(0.5, 1, 1.5),
    algos=("greedy", "random"),
):
    nfleet = np.array(nfleet)
    ratios = np.array(ratios)
    result = pd.DataFrame(
        dict(
            fleet=np.concatenate([nfleet] * (len(ratios) * len(algos))),
            infrastructure=np.concatenate(
                [np.round(nfleet * r).astype(int) for a in algos for r in ratios]
            ),
            ratio=np.concatenate([[r] * len(nfleet) for a in algos for r in ratios]),
            algorithm=np.concatenate(
                [[a] * len(nfleet) for a in algos for r in ratios]
            ),
        )
    )
    return result[["algorithm", "fleet", "infrastructure", "ratio"]]


benchmarks = benchmark_set(nfleet=(50, 100, 200), ratios=(0.8, 1))
benchmarks

Finally, we define a function to run the given algorithm for a given setup:

In [ ]:
def run_benchmark(
    row: pd.Series,
    algos: Optional[Mapping] = None,
    matcher: Optional[Callable] = None,
    repetitions: int = 10,
    **kwargs,
) -> float:
    from timeit import timeit
    from evosim.matchers import socket_compatibility
    from evosim.allocators import random_allocator, greedy_allocator

    fleet = evosim.fleet.random_fleet(row.fleet, **kwargs)
    infrastructure = evosim.charging_posts.random_charging_posts(
        row.infrastructure, **kwargs
    )
    matcher = matcher or socket_compatibility
    algos = algos or dict(
        greedy=partial(greedy_allocator, leaf_size=200), random=random_allocator
    )
    inputs = dict(fleet=fleet, infrastructure=infrastructure, matcher=matcher, **algos)
    return (
        timeit(
            f"{row.algorithm}(fleet, infrastructure, matcher)",
            globals=inputs,
            number=repetitions,
        )
        / repetitions
    )

We can run the benchmarks one after the other with ``pandas.DataFrame.apply``:

In [ ]:
benchmarks["timings"] = benchmarks.apply(partial(run_benchmark, repetitions=3), axis=1)
benchmarks.sample(5)

The results above demonstrate how to benchmark the code. However it is done for a
rather paltry number of cases. Indeed, the code in this notebook runs every time
[EvoSim is modified](https://en.wikipedia.org/wiki/Test-driven_development>), so we
want it to run fairly fast. For the purpose of plotting, lets read a few pre-generated
benchmarks from [file](benchmarks.csv). These benchmarks were run on [1st generation
AMD Epyc 7742](https://en.wikipedia.org/wiki/Epyc) on [Imperial College London's HPC
system](https://www.imperial.ac.uk/admin-services/ict/self-service/research-support/rcs/).

In [ ]:
benchmarks = pd.read_csv("benchmarks.csv")
benchmarks.sample(5)

Finally, lets plot the benchmarks, including second order regressions. [Seaborn](
https://seaborn.pydata.org/) makes it easy to throw a plot together. [Statsmodel](
https://www.statsmodels.org/stable/index.html) could be used to create a more
accurate timing model. Roughly examining the algorithms, we expect the following
scaling:

- the random algorithm is a brute force search to match electric vehicles and
  compatible posts. For a given ratio of fleet size to number of matching posts, it
  should scale as $\mathcal{O}(n^2)$.
- the greedy algorithm combines a $k$ nearest neighbor search to find posts closest to
  each vehicle, and a brute force search for  a compatible post within the $k$ nearest
  neighbpors. The [ball-tree algorithm](https://arxiv.org/abs/1210.6122), and
  specifically, the construction of a ball-tree scales as
  $\mathcal{O}\left(n\log(n)^2\right)$. Hence we expect a scaling of
  $\mathcal{O}\left(k^2n\log(n))^2\right)$.

In [ ]:
def plotting(data: pd.DataFrame):
    import seaborn as sns

    sns.set_theme(style="darkgrid")
    g = sns.lmplot(
        x="fleet", y="timing", order=1, row="ratio", col="algorithm", data=data,
    )
    g = g.set_axis_labels("Fleet Size", "Time (s)")


plotting(benchmarks)

In practice, in this problem setup, non-linear components barely make themselves heard
and the scaling is roughly linear up to 500,000. It would be interesting to check the
influence of the other parameters on running time, such as the greedy algorithm's
`leaf_size` and `nearest_neighbors` options. We also expect that the complexity should
increase were more restrictive matching constraints used, or with fleets and charging
posts which are more difficult to match (e.g. to few of a given kind of post to
accommodate a given kind of vehicle in the fleet).